In [148]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.colors as mcolors

#crea la función DibujaSandKey a la que se le pasa como parametro el nombre de la pestaña del archivo excel y el titulo del grafico
def DibujaSankey(sheet, titulo):
    
    # Preparar datos para el gráfico Sankey
    def format_value(value):
        return f"{value / 1000:.1f}M"
    
    # Leer datos desde el archivo Excel 

    # Ruta del archivo Excel
    file_path = "C:\\Users\\701267061\\Downloads\\EjSandkeydata.xlsx"

    df = pd.read_excel(file_path, sheet_name=sheet, dtype={'x': float, 'y': float}, usecols="A:G")
    
    #crear un dataframe dfEnd con las lineas con destino "END", ASi fijamos coordenadas y colores a Nodos finales
    dfEnd = df.loc[df['target'] == "END"]
    #eliminar del DataFrame original las lineas con destino "END"
    df = df.loc[df['target'] != "END"]

    sources = []
    targets = []
    values = []
    link_colors = []
    node_colors = {}
    node_positions = {}
    
    # Ordenar los nodos únicos según el orden en el DataFrame
    unique_nodes = pd.concat([df['source'], df['target']]).drop_duplicates().tolist()

    for index, row in df.iterrows():
        source = row['source']
        target = row['target']
        value = row['value']
        color = row['color']
        link_color = row['link_color']
        x = row['x']
        y = row['y']
        source_index = unique_nodes.index(source)
        target_index = unique_nodes.index(target)

        sources.append(source_index)
        targets.append(target_index)
        values.append(value)
        link_colors.append(link_color)
        node_colors[source] = color
    
        if not np.isnan(x) and not np.isnan(y):
            node_positions[source] = (x, y)
            
    # para los Nodos que estan en dfEnd, iguala la node_position["Nodo"] a las X e Y del dataframe dfEnd y
    # el node_colors["Nodo"] al color del dataframe dfEnd
    for index, row in dfEnd.iterrows():
        source = row['source']
        x = row['x']
        y = row['y']
        color = row['color']
        if not np.isnan(x) and not np.isnan(y):
            node_positions[source] = (x, y)
        if not pd.isnull(color):
            node_colors[source] = color
            
    node_colors_list = [node_colors.get(node, 'red') for node in unique_nodes]

    # Calcular la suma de valores para cada nodo tanto para los nodos de origen como para los nodos finales
    node_values = {node: sum(df.loc[df['source'] == node, 'value']) for node in unique_nodes}
    for node in unique_nodes:
        if node not in df['source'].tolist():
            node_values[node] += sum(df.loc[df['target'] == node, 'value'])
    
    # Crear gráfico Sankey
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=50,
            thickness=20,
            line=dict(color="black", width=0),
            label=[f"{node} ({format_value(node_values[node])})" for node in unique_nodes],
            color=node_colors_list,
            x=[node_positions[node][0] if node in node_positions else None for node in unique_nodes],
            y=[node_positions[node][1] if node in node_positions else None for node in unique_nodes],

        ),

        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=link_colors
        )
    )])
    # Agregar líneas de cuadrícula horizontales
    # fig.add_trace(go.Scatter(
    #     x=[0, 1],
    #     y=[0, 1],
    #     mode='markers',
    #     marker=dict(opacity=0, size=0),
    #     showlegend=False,
    #     hoverinfo='none'
    # ))
    
    fig.update_layout(title_text=titulo, font_size=10, width=1000, height=600)
    fig.show()

#llama al la funcion DibujaSankey con un par de pestañas con los datos preparados

DibujaSankey ("HojaFake", "BC")


C:\Users\701267061\AppData\Roaming\Python\Python39\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed

